In [35]:
%matplotlib inline
import qutip as qt
import matplotlib.pyplot as plt
import math
import numpy as np

# korisne varijable
ket0 = qt.ket([0])
ket1 = qt.ket([1])
ket00 = qt.tensor(ket0, ket0)
ket11 = qt.tensor(ket1, ket1)
ket01 = qt.tensor(ket0, ket1)
ket10 = qt.tensor(ket1, ket0)
########################################################################


a_in_values = np.linspace(0, 1, 100)
ro_in_values = []
ro_out_values = []

########################################################################
for a_in in a_in_values:
    # b faktor za racunanje alisinog stanja (a je u deklaraciji petlje)
    b_in = math.sqrt(1 - a_in**2)
    #pocetno stanje
    psi_in = a_in * ket0 + b_in * ket1
    ro_in = psi_in * psi_in.dag()
    ro_in_values.append(ro_in)

    #varijabla pomocu koje se odredjuje stanje u kome se nalazi kvantni kanal (ch = channel)
    #u kvantni kanal se nalaze dva uvezana qubita, jedan kod alis, a drugi kod boba,
    #i preko njega se teleportuje alisino stanje (_in)
    theta_ch = math.pi / 4

    psi_ch = math.sin(theta_ch) * ket00 + math.cos(theta_ch) * ket11 #kvantno stanje kanala
    ro_ch = psi_ch * psi_ch.dag() #matrica gustine kanala

    ####matrica gustine koja opisuje celokupno kvantno stanje pocetnog stanja i kvantnog kanala
    ro = qt.tensor(ro_in, ro_ch)

    ##baze po kojima se mere pocetno stanje i jedan qubit iz kvantnog kanala##########
    phi_proj = theta_ch
    sinphi = math.sin(phi_proj)
    cosphi = math.cos(phi_proj)

    b1 = cosphi * ket00 + sinphi * ket11
    b2 = sinphi * ket00 - cosphi * ket11
    b3 = cosphi * ket01 + sinphi * ket10
    b4 = sinphi * ket01 - cosphi * ket10
    #################################################################################

    ###matrice projektivnog merenja (na osnovu baza merenja izracunata u prethodnoj sekciji)
    p1 = (b1 * b1.dag())
    p2 = (b2 * b2.dag())
    p3 = (b3 * b3.dag())
    p4 = (b4 * b4.dag())

    identity = qt.identity(2)

    # kako se ove matrice primenjuju na matricu gustine celokupnog kvantnog stanja, potrebno je da se
    # matrice projekcije pomnoze tenzorski sa identitetom kako bi se poklapale dimenzije (a zbog identiteta, treci qubit
    # ostaje nepromenjen primenjivanjem ovih matrica)
    p1 = qt.tensor(p1, identity)
    p2 = qt.tensor(p2, identity)
    p3 = qt.tensor(p3, identity)
    p4 = qt.tensor(p4, identity)
    ##############################################################################################################

    ###matrice korekcije koje primalac qubita primenjuje kako bi dobio isto kvantno stanje kao pocetno##
    u1 = qt.identity(2)
    u2 = qt.sigmaz()
    u3 = qt.sigmax()
    u4 = qt.sigmaz() * qt.sigmax()
    #####################################################################################################

    ro_out = u1 * (p1 * ro * p1).ptrace(2) * u1.dag()
    ro_out += u2 * (p2 * ro * p2).ptrace(2) * u2.dag()
    ro_out += u3 * (p3 * ro * p3).ptrace(2) * u3.dag()

In [45]:
ro_in_values[65]

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[0.43107846 0.49522704]
 [0.49522704 0.56892154]]

In [44]:
ro_out_values[65]

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[0.43107846 0.49522704]
 [0.49522704 0.56892154]]